In [1]:
import torch
from transformers import Speech2Text2Processor, SpeechEncoderDecoderModel
from datasets import load_dataset
import soundfile as sf

2023-09-29 17:49:39.733682: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-29 17:49:39.760899: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = SpeechEncoderDecoderModel.from_pretrained("facebook/s2t-wav2vec2-large-en-de")
processor = Speech2Text2Processor.from_pretrained("facebook/s2t-wav2vec2-large-en-de")

Config of the decoder: <class 'transformers.models.speech_to_text_2.modeling_speech_to_text_2.Speech2Text2ForCausalLM'> is overwritten by shared decoder config: Speech2Text2Config {
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_cross_attention": true,
  "architectures": [
    "Speech2TextForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "conv_channels": 1024,
  "conv_kernel_sizes": [
    5,
    5
  ],
  "d_model": 256,
  "decoder_attention_heads": 4,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 7,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 4,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "input_channels": 1,
  "input_feat_per_channel": 80,
  "is_decoder": true,
  "is_encoder_decoder": true,
  "max_length"

In [3]:
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

In [4]:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

In [5]:
inputs = processor(ds["speech"][0], sampling_rate=16_000, return_tensors="pt")
generated_ids = model.generate(inputs=inputs["input_values"], attention_mask=inputs["attention_mask"])

In [6]:
transcription = processor.batch_decode(generated_ids)
transcription

['</s> Herr Quilter ist der Apostel der Mittelschicht und wir sind froh, sein Evangelium zu begrüßen. </s>']